In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import avar
import seaborn as sns
# %matplotlib notebook
%matplotlib qt

sns.set()
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'

path = os.getcwd() + "\\coronavirus_data"
csv_files = glob.glob(os.path.join(path, "*.csv"))
data_tot = pd.read_csv(csv_files[0])
# data = data_tot.loc[data['location'] == 'United States']
# data = data_tot.loc[data['location'] == 'Iran']
data = data_tot.loc[data_tot['location'] == 'United States']
data = data.fillna(0).reset_index()
print(data['new_cases'])
# data

0           0.0
1           0.0
2           1.0
3           0.0
4           3.0
         ...   
562    254981.0
563     44082.0
564     24234.0
565    184346.0
566    106871.0
Name: new_cases, Length: 567, dtype: float64


In [8]:
fig = plt.figure()
fig.set_size_inches(6, 4)
ax = fig.add_subplot(2, 1, 1)

ax.plot(data['new_cases'])
ax.set_xlabel('Time [day]')
ax.set_ylabel('Daily new cases')


ct = avar.CharacteristicScale()

ax = fig.add_subplot(2, 1, 2)
ax.plot(ct.allanvar(data['new_cases'].to_numpy()))
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Timescale [# of days]')
ax.set_ylabel('AVAR $\sigma_A^2$')
fig.tight_layout()



In [11]:
from scipy.ndimage import gaussian_filter1d

fig, axs = plt.subplots(2)
horizons = [50, 100, 200]
# horizons = [50]
for h in horizons:
    char = avar.CharacteristicScale(horizon=h)
    
    M_c = []
    for day, v in enumerate(data['new_cases'].to_numpy()):
        char.memory.append(v)
        if day > h:
            davar = char.dynallanvar()
            davar = gaussian_filter1d(davar,2)
            m_c, min_value = char.find_charecteristic_scale(davar)
            M_c.append(m_c)
        else:
            M_c.append(1)
            
    axs[1].plot(M_c, label='horizon = '+str(h),lw=1)
    axs[1].set_xlabel('Time [day]')
    axs[1].set_ylabel('Timescale [# of days]')
    axs[1].set_yscale("log")
    axs[1].legend()
axs[0].plot(data['new_cases'].to_numpy(),lw=1)
axs[0].set_xlabel('Time [day]')
axs[0].set_ylabel('Daily new cases')
fig.tight_layout()
plt.show()